# Data Tranformation

In [19]:
import pandas as pd
import numpy as np
import requests

In [2]:
repo_url = "https://raw.githubusercontent.com/baertsch/MGT-502-ML-Project/main/"
items_df = pd.read_csv(repo_url + "kaggle_data/items.csv")
display(items_df.head())

,Title,Author,ISBN Valid,Publisher,Subjects,i
0,Classification décimale universelle : édition ...,NaN,9782871303336; 2871303339,Ed du CEFAL,Classification décimale universelle; Indexatio...,0
1,Les interactions dans l'enseignement des langu...,"Cicurel, Francine, 1947-",9782278058327; 2278058320,Didier,didactique--langue étrangère - enseignement; d...,1
2,Histoire de vie et recherche biographique : pe...,NaN,2343190194; 9782343190198,L'Harmattan,Histoires de vie en sociologie; Sciences socia...,2
3,Ce livre devrait me permettre de résoudre le c...,"Mazas, Sylvain, 1980-",9782365350020; 236535002X; 9782365350488; 2365...,Vraoum!,Moyen-Orient; Bandes dessinées autobiographiqu...,3
4,Les années glorieuses : roman /,"Lemaitre, Pierre, 1951-",9782702180815; 2702180817; 9782702183618; 2702...,Calmann-Lévy,France--1945-1975; Roman historique; Roman fra...,4


In [3]:
items_df.isna().sum()

Title            0
Author        2653
ISBN Valid     723
Publisher       25
Subjects      2223
i                0
dtype: int64

In [14]:
#items_df[items_df[items_df['Author'].isna()] & items_df[items_df['ISBN Valid'].isna()]]
#items_df[items_df['ISBN Valid'].isna()]
items_df[items_df.duplicated(subset=['Title', 'Author'], keep=False)].sort_values('Title')

,Title,Author,ISBN Valid,Publisher,Subjects,i
12948,"""Folâtrer avec les démons"" : sabbat et chasse ...","Ostorero, Martine",2940110050; 9782940110056,Dupuis; Université de Lausanne/Faculté des let...,"procès--sorcellerie--Vevey (Suisse, VD)--1448-...",12948
4971,"""Folâtrer avec les démons"" : sabbat et chasse ...","Ostorero, Martine",2940110611; 9782940110612,Université de Lausanne/Faculté des lettres/Sec...,"Sorcellerie--Procès--Vevey (Suisse, VD)--15e s...",4971
12360,1352 : un médecin contre la tyrannie : roman h...,"Favre, Philippe, enseignant",9782828921439; 2828921433,Editions Favre,NaN,12360
13958,1352 : un médecin contre la tyrannie : roman h...,"Favre, Philippe, enseignant",9782828914097; 2828914097,Favre,"Amédée--(comte de Savoie ;--6); Perronet, Guil...",13958
904,14/18 : la Suisse et la Grande Guerre /,NaN,9783039193424; 3039193422,Hier und jetzt,Guerre mondiale (1914-1918); Suisse; [Études d...,904
...,...,...,...,...,...,...
12528,Vies des douze Césars /,"Suetonius Tranquillus, Caius",NaN,Les Belles Lettres,NaN,12528
10253,Vies.,Plutarque 0046?-0120?,NaN,Les Belles Lettres,NaN,10253
13641,Vies.,Plutarque 0046?-0120?,NaN,Les Belles Lettres,NaN,13641
14964,À la ligne : feuillets d'usine /,"Ponthus, Joseph",9782072881862; 2072881862,Gallimard,NaN,14964


In [20]:
# Function to fetch metadata from OpenLibrary API for a given ISBN
def fetch_metadata(isbn):
    url = f"https://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&format=json&jscmd=data"
    response = requests.get(url).json()
    book_data = response.get(f"ISBN:{isbn}", {})
    
    # Set author and title to NaN if not found
    author_name = book_data.get("authors", [{}])[0].get("name", np.nan)
    book_title = book_data.get("title", np.nan)
    
    return author_name, book_title

# Iterate through rows with missing values and fill them using the API
for index, row in items_df[items_df['Author'].isna() | items_df['Title'].isna()].iterrows():
    isbn = row.get('ISBN')  # Replace 'ISBN' with the actual column name for ISBN in your DataFrame
    if pd.notna(isbn):  # Ensure ISBN is not NaN
        author, title = fetch_metadata(isbn)
        if pd.isna(row['Author']):
            items_df.at[index, 'Author'] = author
        if pd.isna(row['Title']):
            items_df.at[index, 'Title'] = title

# Display the updated DataFrame
display(items_df)

,Title,Author,ISBN Valid,Publisher,Subjects,i
0,Classification décimale universelle : édition ...,NaN,9782871303336; 2871303339,Ed du CEFAL,Classification décimale universelle; Indexatio...,0
1,Les interactions dans l'enseignement des langu...,"Cicurel, Francine, 1947-",9782278058327; 2278058320,Didier,didactique--langue étrangère - enseignement; d...,1
2,Histoire de vie et recherche biographique : pe...,NaN,2343190194; 9782343190198,L'Harmattan,Histoires de vie en sociologie; Sciences socia...,2
3,Ce livre devrait me permettre de résoudre le c...,"Mazas, Sylvain, 1980-",9782365350020; 236535002X; 9782365350488; 2365...,Vraoum!,Moyen-Orient; Bandes dessinées autobiographiqu...,3
4,Les années glorieuses : roman /,"Lemaitre, Pierre, 1951-",9782702180815; 2702180817; 9782702183618; 2702...,Calmann-Lévy,France--1945-1975; Roman historique; Roman fra...,4
...,...,...,...,...,...,...
15286,Le vagabond de Tokyo /,"Fukutani, Takashi, 1952-2000",9782353480111; 235348011X; 9782353480241; 2353...,Le Lézard noir,Mangas,15286
15287,God of high school : le match contre les dieux /,"Park, Yong-Je",9782382880203; 2382880201; 9782382880210; 2382...,Kbooks,NaN,15287
15288,Blue Lock /,"Kaneshiro, Muneyuki",9782811650254; 2811650253; 9782811661274; 2811...,Pika,Compétitions; Football; Entraînement (sports);...,15288
15289,Red eyes sword : akame ga kill ! Zero /,Takahiro,9782368522134; 2368522131; 9782368522141; 2368...,Kurokawa,Bandes dessinées; Mangas,15289
